#Prepare data from snOPY database

##Goal

In order to unify our calculations of snoRNA target sites we decided to change database from [snoRNABase](https://www-snorna.biotoul.fr/) to [snOPY](http://snoopy.med.miyazaki-u.ac.jp/snorna_db.cgi). This is because snoRNABase does not have any annotation of mouse snoRNAs and it is a bit outdated. We kindly obtained xml files with database entries from dr Kenmochi from Frontier Science Research Center, University of Miyazaki, Japan. With this in hand we would be able to build an uniform pipeline for both species.

The biggest disadvantage of this database is that the coordinates are mixed i.e. for human coordinates consist of hg19 and hg18 assemblies so we need to identify them and lift over hg18 to hg19. The same is might be true for mouse were there is a mix of mm9 and mm10 coordinates.

##Materials and Methods

###Read and wrangle the data

An example of XML input file is shown below:

In [ ]:
<snoopy id='Homo_sapiens300000'>
<dataquality>B</dataquality>
<openstatus>+</openstatus>
<lastupdate>2009-07-01</lastupdate>
<blindnote>
gene name:AC013603.17->SNORA70B-pseud.

2009年7月1日（山元）gene name：SNORA70B-pseud→SNORA70BL3


</blindnote>
<organism>Homo_sapiens</organism>
<gene_name>SNORA70BL3</gene_name>
<family_name></family_name>
<alias></alias>
<family_base>Homo_sapiens</family_base>
<chrom_or_contig>8</chrom_or_contig>
<genome_position>33517083..33517197</genome_position>
<strand>+</strand>
<box type='c'></box>
<box type='d'></box>
<box type='h'>4..9</box>
<box type='aca'>110..112</box>
<host_id>Homo_sapiens100000</host_id>
<host_gene>Mono:286:AC013603.17</host_gene>
<host_intron></host_intron>
<host_position>1001..1115</host_position>
<organization>Mono</organization>
<mod_type>H/ACA</mod_type>
<mod_rna></mod_rna>
<mod_site></mod_site>
<duplex_region></duplex_region>
<accession></accession>
<evidence></evidence>
<note></note>
<sequence>
TTGATAAGAAAGAAGTTCTGATCCCAGTGTGCAATGGCTGCAAACAGCAGCTTTCTTGGT
GGTGTACATGGCCTGTTTCTTGTATGGGTTGCTCTAAGGGTCCTTGGAGACAGGC
</sequence>
</snoopy>

In [5]:
data = {}
for xml in glob.glob("./snOPY/snoRNAs/*.xml"):
    try:
        tree = ET.parse(xml)
        root = tree.getroot()
    except ET.ParseError, e:
        print "Cannot parse: ", xml, e
    tmp = {}
    for child in root:
        if child.tag == "box":
            if child.text:
                tmp[child.tag + "_" + child.attrib['type']] = child.text
            else:
                tmp[child.tag] = np.nan
        else:
            if child.text:
                tmp[child.tag] = child.text.replace("\n", "")
            else:
                tmp[child.tag] = np.nan
    data[root.attrib["id"]] = tmp

In [6]:
df = pd.DataFrame(data).T
df["id"] = [i.split("_")[-1] for i in df.index.tolist()]
df = df.dropna(subset=["mod_type", "genome_position"])
df["start"] = [int(i.split("..")[0]) - 1 for i in df["genome_position"]]
df["end"] = [int(i.split("..")[1]) for i in df["genome_position"]]
df["snor_id"] = df.gene_name + "_" + df.id
df = df[["chrom_or_contig",
    "start",
    "end",
    "snor_id",
    "mod_type",
    "strand",
    "sequence",
    "box_d",
    "box_c",
    "box_h",
    "box_aca",
    "alias",
    "gene_name",
    "accession",
    "mod_site",
    "host_gene",
    "host_id",
    "organization",
    "organism",
    "note"]]
df.mod_type =df.mod_type.str.replace(" Pu", "")
df.head()

,chrom_or_contig,start,end,snor_id,mod_type,strand,sequence,box_d,box_c,box_h,box_aca,alias,gene_name,accession,mod_site,host_gene,host_id,organization,organism,note
Homo_sapiens300000,8,33517082,33517197,SNORA70BL3_sapiens300000,H/ACA,+,TTGATAAGAAAGAAGTTCTGATCCCAGTGTGCAATGGCTGCAAACA...,NaN,NaN,4..9,110..112,NaN,SNORA70BL3,NaN,NaN,Mono:286:AC013603.17,Homo_sapiens100000,Mono,Homo_sapiens,NaN
Homo_sapiens300001,12,55541425,55541559,SNORA48L6_sapiens300001,H/ACA,+,TGTTCCCGATCTGGTAGAGTAGCATCTGGTTGGTGGTGACCATCTA...,NaN,NaN,16..21,129..131,NaN,SNORA48L6,NaN,NaN,Mono:245:AC121758.14,Homo_sapiens100001,Mono,Homo_sapiens,NaN
Homo_sapiens300002,2,139985463,139985595,SNORA72L7_sapiens300002,H/ACA,+,CTGTGAATATTCTCTGTGTTCTGATTATGTGAGAGCCAGGACAGGC...,NaN,NaN,59..64,59..61,NaN,SNORA72L7,NaN,NaN,Mono:45:AC016710.7,Homo_sapiens100002,Mono,Homo_sapiens,NaN
Homo_sapiens300003,7,12706907,12707039,SNORA64L2_sapiens300003,H/ACA,-,ACACTCTCAGCTATGTATAGTTACACCTGGCTTCACTTGTGTGACT...,NaN,NaN,65..70,127..129,NaN,SNORA64L2,NaN,NaN,Mono:182:AC011891.3,Homo_sapiens100003,Mono,Homo_sapiens,NaN
Homo_sapiens300004,19,9791642,9791770,SNORA70L2_sapiens300004,H/ACA,+,CTGAATTGGCAAGTCCACCAAGAAACTATAAAAATGTGTGTGTGCA...,NaN,NaN,23..28,123..125,NaN,SNORA70L2,NaN,NaN,Mono:137:AC008752.6,Homo_sapiens100004,Mono,Homo_sapiens,NaN


###Write columns help

In [161]:
desc_map = OrderedDict((("Chromosome", "Chromosome on which gene is located"),
            ("Start", "0-based start coordinate"),
            ("End", "1-based end coordinate"),
            ("snoRNA ID", "ID for snoRNA"),
            ("Type", "Type of snoRNA (C/D or H/ACA)"),
            ("Strand", "Strand on which gene is located"),
            ("Sequence", "Sequence of snoRNA"),
            ("D-boxes", "1-based coordinates snoRNA D-boxes in form of "+
                       "D-box_start..D-box_end or D'-box_start..D'-box_end,D-box_start..D-box_end "+
                       "eg. 60..63 or 30..33,60..63. It must be 4 nucleotides long."),
            ("C-boxes", "Same as above (not that prime boxes are reversed in positions) "+
                       "with exception of the length constraint"),
            ("H-box", "1-based position of the H box in H/ACA box snoRNAs in form of start..end"),
            ("ACA-box", "1-based position of the ACA box in H/ACA box snoRNA in form of start..end"),
            ("Alias", "Alias/Alternative name for snoRNA"),
            ("Gene name", "Name of the snoRNA gene"),
            ("Accession", "Accession (NCBI, RefSeq etc) for snoRNA"),
            ("Modification sites", "Coma separated list of modified sites in form of "+
                                  "ModifiedRNA:NucleotidePosition eg. 28S:U46,18S:G52"),
            ("Host gene", "Host gene name if any"),
            ("Host ID", "Host locus ID"),
            ("Organization", "Organisation of the gene eg. intronic"),
            ("Organism", "Species"),
            ("Note", "Additional info about snoRNA"))
)
with open("Columns_description.tab", "w") as col_desc:
    counter = 1
    for col, desc in desc_map.iteritems():
        col_desc.write("%i. %s\t%s\n" % (counter,
                                         col,
                                         desc))
        counter += 1

###Homo sapiens

####Lift over hg18 coordinates

The procedure of this is little bit tedious. First we have to check which assembly it is. Fortunately we have sequences of snoRNAs so we can use local alignment to check it.

In [162]:
to_save = ["chrom_or_contig",
           "start",
           "end",
           "snor_id",
           "mod_type",
           "strand",
           "sequence"]
homo_sapiens = df[(df.organism == "Homo_sapiens")]
homo_sapiens.chrom_or_contig = "chr" + homo_sapiens.chrom_or_contig
homo_sapiens[to_save].to_csv("human/hsa_snoRNAs.bed",
                             header=None,
                             index=None,
                             sep="\t")

After the data preparation we extract sequences from hg19 and hg18 and append them to the previously generated file. In the end we read generated file into variable.

In [163]:
%%bash
rg-extract-sequences.py \
    --input human/hsa_snoRNAs.bed \
    --output human/hsa_snoRNAs.bed.hg19 \
    --genome-dir human/hg19/
rg-extract-sequences.py \
    --input human/hsa_snoRNAs.bed.hg19 \
    --output human/hsa_snoRNAs.bed.hg19.hg18 \
    --genome-dir /import/bc2/home/zavolan/GROUP/DB/UCSC/hg18/

In [164]:
for_assembly_human = pd.read_table("human/hsa_snoRNAs.bed.hg19.hg18",
                             header=None)

In [165]:
for_assembly_human.head()

,0,1,2,3,4,5,6,7,8
0,chrY,26802917,26803056,SNORA70BL6_sapiens300100,H/ACA,+,AGATCAGCTTTTCAAAGGTTTACTGCATTTTACCCTCCCTGATGGG...,attgtgggttgccagggtcacacatgcaattactgatttactgggt...,agatcagcttttcaaaggtttaCTGCATTTTACCCTCCCTGATGGG...
1,chrY,16759521,16759653,SNORA20L4_sapiens300149,H/ACA,-,CTTCCCATTTATTTGCTGGTATAATGATACAATGATATGAGCAGTT...,AATGACAACTGATTAATTTTGTTAATAGTAATGGAAGGTTTTCTTT...,CTTCCCATTTATTTGCTGGTATAATGATACAATGATATGAGCAGTT...
2,chrY,23978633,23978772,SNORA70BL5_sapiens300155,H/ACA,-,AGATCAGCTTTTCAAAGGTTTACTGCATTTTACCCTCCCTGATGGG...,TTGTGAAAAAGGCTGGCTGTCAAAACCCACTTCAAGACCCTAAAAG...,agatcagcttttcaaaggtttaCTGCATTTTACCCTCCCTGATGGG...
3,chrX,86288391,86288463,SNORD45BL2_sapiens300006,C/D,+,GGTCAATGATGTAATGGCATGTATTAGCTGAATCCAAAGTTGAAGT...,agttcccagacagggtcgcggcggggcagaggcgctccccacatcc...,ggTCAATGATGTAATGGCATGTATTAGCTGAATCCAAAGTTGAAGT...
4,chrX,3460157,3460292,SNORA48L4_sapiens300050,H/ACA,-,TTTCCCTGACCTGGGTAGAGTGGCATCCAGTTGGTGGTGCCCATCT...,ATAGTCTCTGCACCCATGAAAAGTTCTTGCCCACATGTCCGATCAG...,TTTCCCTGACCTGGGTAGAGTGGCATCCAGTTGGTGGTGCCCATCT...


#####Append assembly

Now we check from which assembly the sequence is generated and with this we can assign the assembly to snoRNAs.

In [166]:
for_assembly_human = apped_assembly(for_assembly_human)

In [167]:
for_assembly_human.head()

,0,1,2,3,4,5,6,7,8,hg19score,hg18score,assembly
0,chrY,26802917,26803056,SNORA70BL6_sapiens300100,H/ACA,+,AGATCAGCTTTTCAAAGGTTTACTGCATTTTACCCTCCCTGATGGG...,attgtgggttgccagggtcacacatgcaattactgatttactgggt...,agatcagcttttcaaaggtttaCTGCATTTTACCCTCCCTGATGGG...,15,278,hg18
1,chrY,16759521,16759653,SNORA20L4_sapiens300149,H/ACA,-,CTTCCCATTTATTTGCTGGTATAATGATACAATGATATGAGCAGTT...,AATGACAACTGATTAATTTTGTTAATAGTAATGGAAGGTTTTCTTT...,CTTCCCATTTATTTGCTGGTATAATGATACAATGATATGAGCAGTT...,15,264,hg18
2,chrY,23978633,23978772,SNORA70BL5_sapiens300155,H/ACA,-,AGATCAGCTTTTCAAAGGTTTACTGCATTTTACCCTCCCTGATGGG...,TTGTGAAAAAGGCTGGCTGTCAAAACCCACTTCAAGACCCTAAAAG...,agatcagcttttcaaaggtttaCTGCATTTTACCCTCCCTGATGGG...,14,278,hg18
3,chrX,86288391,86288463,SNORD45BL2_sapiens300006,C/D,+,GGTCAATGATGTAATGGCATGTATTAGCTGAATCCAAAGTTGAAGT...,agttcccagacagggtcgcggcggggcagaggcgctccccacatcc...,ggTCAATGATGTAATGGCATGTATTAGCTGAATCCAAAGTTGAAGT...,8,144,hg18
4,chrX,3460157,3460292,SNORA48L4_sapiens300050,H/ACA,-,TTTCCCTGACCTGGGTAGAGTGGCATCCAGTTGGTGGTGCCCATCT...,ATAGTCTCTGCACCCATGAAAAGTTCTTGCCCACATGTCCGATCAG...,TTTCCCTGACCTGGGTAGAGTGGCATCCAGTTGGTGGTGCCCATCT...,18,270,hg18


A quick look on the counts for each assembly assures us that most of the snoRNAs are derived from hg18. In this case we need to lift them over.

In [168]:
pd.DataFrame(for_assembly_human.groupby("assembly").size())

,0
assembly,
ambiguous,1
hg18,515
hg19,245


#####Actual lift over

Now we are ready to save the data and lift over coordinates from that need to be lifted.

In [169]:
for_assembly_human[for_assembly_human.assembly=='hg18'].ix[:,:5].to_csv(
        "human/hsa_snoRNAs_to_lift_over.bed",
        sep="\t",
        header=None,
        index=None)

In [170]:
%%bash
CrossMap.py bed genome_conversion_files/hg18ToHg19.over.chain.gz \
                human/hsa_snoRNAs_to_lift_over.bed |\
                cut -f 8,9,10,11,12,13,14 > \
                human/hsa_snoRNAs_to_lift_over.bed.lifted

@ 2015-03-13 15:24:31: Read chain_file:  genome_conversion_files/hg18ToHg19.over.chain.gz


In [171]:
human_lifted = pd.read_table("human/hsa_snoRNAs_to_lift_over.bed.lifted",
                             header=None)
human_lifted.index = ["Homo_%s" % i.split("_")[-1] for i in human_lifted[3]]
for_assembly_human.index = ["Homo_%s" % i.split("_")[-1] 
                                for i in for_assembly_human[3]]

Quick look on all the datasets:

In [172]:
human_lifted.head()

,0,1,2,3,4,5
Homo_sapiens300100,chrY,28393529,28393668,SNORA70BL6_sapiens300100,H/ACA,+
Homo_sapiens300149,chrY,18250127,18250259,SNORA20L4_sapiens300149,H/ACA,-
Homo_sapiens300155,chrY,25569245,25569384,SNORA70BL5_sapiens300155,H/ACA,-
Homo_sapiens300006,chrX,86401735,86401807,SNORD45BL2_sapiens300006,C/D,+
Homo_sapiens300050,chrX,3450157,3450292,SNORA48L4_sapiens300050,H/ACA,-


In [173]:
for_assembly_human.head()

,0,1,2,3,4,5,6,7,8,hg19score,hg18score,assembly
Homo_sapiens300100,chrY,26802917,26803056,SNORA70BL6_sapiens300100,H/ACA,+,AGATCAGCTTTTCAAAGGTTTACTGCATTTTACCCTCCCTGATGGG...,attgtgggttgccagggtcacacatgcaattactgatttactgggt...,agatcagcttttcaaaggtttaCTGCATTTTACCCTCCCTGATGGG...,15,278,hg18
Homo_sapiens300149,chrY,16759521,16759653,SNORA20L4_sapiens300149,H/ACA,-,CTTCCCATTTATTTGCTGGTATAATGATACAATGATATGAGCAGTT...,AATGACAACTGATTAATTTTGTTAATAGTAATGGAAGGTTTTCTTT...,CTTCCCATTTATTTGCTGGTATAATGATACAATGATATGAGCAGTT...,15,264,hg18
Homo_sapiens300155,chrY,23978633,23978772,SNORA70BL5_sapiens300155,H/ACA,-,AGATCAGCTTTTCAAAGGTTTACTGCATTTTACCCTCCCTGATGGG...,TTGTGAAAAAGGCTGGCTGTCAAAACCCACTTCAAGACCCTAAAAG...,agatcagcttttcaaaggtttaCTGCATTTTACCCTCCCTGATGGG...,14,278,hg18
Homo_sapiens300006,chrX,86288391,86288463,SNORD45BL2_sapiens300006,C/D,+,GGTCAATGATGTAATGGCATGTATTAGCTGAATCCAAAGTTGAAGT...,agttcccagacagggtcgcggcggggcagaggcgctccccacatcc...,ggTCAATGATGTAATGGCATGTATTAGCTGAATCCAAAGTTGAAGT...,8,144,hg18
Homo_sapiens300050,chrX,3460157,3460292,SNORA48L4_sapiens300050,H/ACA,-,TTTCCCTGACCTGGGTAGAGTGGCATCCAGTTGGTGGTGCCCATCT...,ATAGTCTCTGCACCCATGAAAAGTTCTTGCCCACATGTCCGATCAG...,TTTCCCTGACCTGGGTAGAGTGGCATCCAGTTGGTGGTGCCCATCT...,18,270,hg18


In [174]:
homo_sapiens.head()

,chrom_or_contig,start,end,snor_id,mod_type,strand,sequence,box_d,box_c,box_h,box_aca,alias,gene_name,accession,mod_site,host_gene,host_id,organization,organism,note
Homo_sapiens300000,chr8,33517082,33517197,SNORA70BL3_sapiens300000,H/ACA,+,TTGATAAGAAAGAAGTTCTGATCCCAGTGTGCAATGGCTGCAAACA...,NaN,NaN,4..9,110..112,NaN,SNORA70BL3,NaN,NaN,Mono:286:AC013603.17,Homo_sapiens100000,Mono,Homo_sapiens,NaN
Homo_sapiens300001,chr12,55541425,55541559,SNORA48L6_sapiens300001,H/ACA,+,TGTTCCCGATCTGGTAGAGTAGCATCTGGTTGGTGGTGACCATCTA...,NaN,NaN,16..21,129..131,NaN,SNORA48L6,NaN,NaN,Mono:245:AC121758.14,Homo_sapiens100001,Mono,Homo_sapiens,NaN
Homo_sapiens300002,chr2,139985463,139985595,SNORA72L7_sapiens300002,H/ACA,+,CTGTGAATATTCTCTGTGTTCTGATTATGTGAGAGCCAGGACAGGC...,NaN,NaN,59..64,59..61,NaN,SNORA72L7,NaN,NaN,Mono:45:AC016710.7,Homo_sapiens100002,Mono,Homo_sapiens,NaN
Homo_sapiens300003,chr7,12706907,12707039,SNORA64L2_sapiens300003,H/ACA,-,ACACTCTCAGCTATGTATAGTTACACCTGGCTTCACTTGTGTGACT...,NaN,NaN,65..70,127..129,NaN,SNORA64L2,NaN,NaN,Mono:182:AC011891.3,Homo_sapiens100003,Mono,Homo_sapiens,NaN
Homo_sapiens300004,chr19,9791642,9791770,SNORA70L2_sapiens300004,H/ACA,+,CTGAATTGGCAAGTCCACCAAGAAACTATAAAAATGTGTGTGTGCA...,NaN,NaN,23..28,123..125,NaN,SNORA70L2,NaN,NaN,Mono:137:AC008752.6,Homo_sapiens100004,Mono,Homo_sapiens,NaN


Now we will change coordinates in the main DataFrame if they are hg18.

In [175]:
homo_sapiens_with_assembly = homo_sapiens.join(for_assembly_human['assembly'])

In [178]:
new_chrom = []
new_start = []
new_end = []
for idx, row in homo_sapiens_with_assembly.iterrows():
    if row.assembly == "hg19" or row.assembly == 'ambiguous':
        new_chrom.append(row.chrom_or_contig)
        new_start.append(row.start)
        new_end.append(row.end)
    elif row.assembly == "hg18":
        new_chrom.append(human_lifted.ix[idx, 0])
        new_start.append(human_lifted.ix[idx, 1])
        new_end.append(human_lifted.ix[idx, 2])
    else:
        print row
        raise Exception("Do not know assembly named %s" % row.assembly)
homo_sapiens_with_assembly.chrom_or_contig = new_chrom
homo_sapiens_with_assembly.start = new_start
homo_sapiens_with_assembly.end = new_end

In [179]:
del homo_sapiens_with_assembly["assembly"]

#####Check if there is any strange annotation

Just quick look if there is anything strange in the annotations. (Both is not strange)

In [180]:
homo_sapiens_with_assembly[(homo_sapiens_with_assembly.mod_type!="C/D") & 
                          (homo_sapiens_with_assembly.mod_type!="H/ACA")]

,chrom_or_contig,start,end,snor_id,mod_type,strand,sequence,box_d,box_c,box_h,box_aca,alias,gene_name,accession,mod_site,host_gene,host_id,organization,organism,note
Homo_sapiens300786,chr12,6619387,6619717,SCARNA10_sapiens300786,Both,+,GCCACATGATGATATCAAGGCTGTTGTGATTCAGTTGGTTTGGCTA...,322..325,7..12,131..136,NaN,U85,SCARNA10,"NR_004387,NT_009759","U5:U46,U5:C45",NCAPD2,Homo_sapiens100543,Intronic,Homo_sapiens,NaN


####Save snoRNAs

In [183]:
homo_sapiens_cd = homo_sapiens_with_assembly[homo_sapiens_with_assembly.mod_type.str.contains("C/D")]
homo_sapiens_cd = homo_sapiens_cd.dropna(subset=["box_d"])
homo_sapiens_aca = homo_sapiens_with_assembly[homo_sapiens_with_assembly.mod_type.str.contains("H/ACA")]
homo_sapiens_aca = homo_sapiens_aca.dropna(subset=["box_h"])

In [184]:
homo_sapiens_cd.to_csv("human/hsa_cd_box_snoRNAs.tab",
                       index=False,
                       header=False,
                       sep="\t",
                       na_rep="NaN")
homo_sapiens_aca.to_csv("human/hsa_haca_box_snoRNAs.tab",
                       index=False,
                       header=False,
                       sep="\t",
                       na_rep="NaN",
                        encoding='utf-8')

###Mus musculus

Whole procedure should be repeated for mouse too.

In [7]:
to_save = ["chrom_or_contig",
           "start",
           "end",
           "snor_id",
           "mod_type",
           "strand",
           "sequence"]
mus_musculus = df[(df.organism == "Mus_musculus")]
mus_musculus.chrom_or_contig = "chr" + mus_musculus.chrom_or_contig
mus_musculus.mod_type = mus_musculus.mod_type.str.replace(" Pu", "")
mus_musculus[to_save].to_csv("mouse/mmu_snoRNAs.bed",
                             header=None,
                             index=None,
                             sep="\t")

/import/bc2/apps/Python/2.7.5-goolf-1.4.10/lib/python2.7/site-packages/pandas-0.15.2-py2.7-linux-x86_64.egg/pandas/core/generic.py:1974: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [190]:
%%bash
rg-extract-sequences.py \
    --input mouse/mmu_snoRNAs.bed \
    --output mouse/mmu_snoRNAs.bed.mm10 \
    --genome-dir mouse/mm10/
rg-extract-sequences.py \
    --input mouse/mmu_snoRNAs.bed.mm10 \
    --output mouse/mmu_snoRNAs.bed.mm10.mm9 \
    --genome-dir /import/bc2/home/zavolan/GROUP/DB/UCSC/mm9/

Cannot read from mouse/mm10/.fa file


In [192]:
for_assembly_mouse = pd.read_table("mouse/mmu_snoRNAs.bed.mm10.mm9",
                             header=None)

In [196]:
for_assembly_mouse = apped_mouse_assembly(for_assembly_mouse)

In [197]:
for_assembly_mouse.head()

,0,1,2,3,4,5,6,7,8,mm10score,mm9score,assembly
0,chr7,113570225,113570361,SNORA17_musculus300011,H/ACA,+,TCTGCCCCAGGAGCATTGCAGGTATGACTGCTGTGGCACATGTGTG...,AGAAAGGTAATGAATAAATGATGCTCATCCTAGAATGTATAATTTT...,TCTGCCCCAGGAGCATTGCAGGTATGACTGCTGTGGCACATGTGTG...,12,272,mm9
1,chr7,112790851,112790990,SNORA43_musculus300020,H/ACA,-,GCTGTCCTAGATCAGCACCACAGACCTGGCCTAGCTGCTCAGCCCT...,CCTCTACTGAAGAAACACTATAATGTCTGCCTTCTCACTGAGAGGG...,GCTGTCCTAGATCAGCACCACAGACCTGGCCTAGCTGCTCAGCCCT...,12,278,mm9
2,chr7,67004099,67004193,SNORD116_musculus300055,C/D,-,GGATCTATGATGATTCCCAGTCAAACATTCCTTGGAAAAGCTGAAC...,TGAGTGCTTTTACTATTATGGCTAAATGTGGGTCTCTTCTCTGAAG...,GGATCTATGATGATTCCCAGTCAAACATTCCTTGGAAAAGCTGAAC...,14,188,mm9
3,chr7,67006616,67006710,SNORD116_musculus300056,C/D,-,GGATCTATGATGATTCCCAGTCAAACATTCCTTGGAAAAGCTGAAC...,ttctgttagcatgccttccatcatgatggactagttctgaaataag...,GGATCTATGATGATTCCCAGTCAAACATTCCTTGGAAAAGCTGAAC...,12,188,mm9
4,chr7,67009125,67009219,SNORD116_musculus300057,C/D,-,GGATCTATGATGATTCCCAGTCAAACATTCCTTGGAAAAGCTGAAC...,GAGGGTTGTGGGATGTAGGCAGACTCACACCTAAATGAATCATGTC...,GGATCTATGATGATTCCCAGTCAAACATTCCTTGGAAAAGCTGAAC...,16,188,mm9


In [198]:
pd.DataFrame(for_assembly_mouse.groupby("assembly").size())

,0
assembly,
ambiguous,8
mm9,965


The results above shows that all the coordinates for mouse are from mm9 assembly. This is good sign. We do not need to go though all the steps of remapping - we will only lift the coordinates over.

####Lifting over

In [8]:
mus_musculus_cd = df[(df.organism == "Mus_musculus") & (df.mod_type=="C/D")]
mus_musculus_cd = mus_musculus_cd.dropna(subset=["box_d"])
mus_musculus_aca = df[(df.organism == "Mus_musculus") & (df.mod_type=="H/ACA")]
mus_musculus_aca = mus_musculus_aca.dropna(subset=["box_h"])

aca_for_conversion = df[(df.organism == "Mus_musculus") & (df.mod_type=="H/ACA")]
aca_for_conversion["chrom_or_contig"] = "chr" + aca_for_conversion.chrom_or_contig
aca_for_conversion.iloc[:,:6].to_csv("mouse/mmu_mm9_haca_box_snoRNAs.bed",
                                    header=None,
                                    sep="\t",
                                    index=None)

cd_for_conversion = df[(df.organism == "Mus_musculus") & (df.mod_type=="C/D")]
cd_for_conversion["chrom_or_contig"] = "chr" + cd_for_conversion.chrom_or_contig
cd_for_conversion.iloc[:,:6].to_csv("mouse/mmu_mm9_cd_box_snoRNAs.bed",
                                    header=None,
                                    sep="\t",
                                    index=None)

/import/bc2/apps/Python/2.7.5-goolf-1.4.10/lib/python2.7/site-packages/IPython/kernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/import/bc2/apps/Python/2.7.5-goolf-1.4.10/lib/python2.7/site-packages/IPython/kernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


####Convert mm9 coordinates to mm10

In [9]:
%%bash
CrossMap.py bed ~/Data/Genome/genome_conversion_files/mm9ToMm10.over.chain.gz \
                mouse/mmu_mm9_cd_box_snoRNAs.bed | \
                cut -f 8,9,10,11,12,13,14 > \
                mouse/mmu_mm10_cd_box_snoRNAs.bed
CrossMap.py bed ~/Data/Genome/genome_conversion_files/mm9ToMm10.over.chain.gz \
                mouse/mmu_mm9_haca_box_snoRNAs.bed | \
                cut -f 8,9,10,11,12,13,14 > \
                mouse/mmu_mm10_haca_box_snoRNAs.bed

@ 2015-04-10 15:28:14: Read chain_file:  /import/bc2/home/zavolan/gumiennr/Data/Genome/genome_conversion_files/mm9ToMm10.over.chain.gz
End corrdinate is not an integer. skip 166888972	166889065	SNORD12_musculus300988	C/D	+
@ 2015-04-10 15:28:16: Read chain_file:  /import/bc2/home/zavolan/gumiennr/Data/Genome/genome_conversion_files/mm9ToMm10.over.chain.gz


In [16]:
cd_mm9_2_mm10 = pd.read_table("mouse/mmu_mm10_cd_box_snoRNAs.bed", index_col=3,
                           header=None)
aca_mm9_2_mm10 = pd.read_table("mouse/mmu_mm10_haca_box_snoRNAs.bed", index_col=3,
                           header=None)

In [12]:
cd_mm9_2_mm10.head()

,0,1,2,4,5
3,,,,,
SNORD86_musculus300006,chr14,56837070,56837156,C/D,-
SNORD23_musculus300008,chr10,21878783,21878892,C/D,-
SNORND104_musculus300032,chr11,106500992,106501062,C/D,+
SNORD116_musculus300055,chr7,59859213,59859307,C/D,-
SNORD116_musculus300056,chr7,59861730,59861824,C/D,-


In [17]:
mus_musculus_cd["start"] = [int(cd_mm9_2_mm10.ix[i, 1]) if i in cd_mm9_2_mm10.index
                            else np.nan for i in mus_musculus_cd.snor_id]
mus_musculus_cd["end"] = [int(cd_mm9_2_mm10.ix[i, 2]) if i in cd_mm9_2_mm10.index
                            else np.nan for i in mus_musculus_cd.snor_id]
mus_musculus_cd = mus_musculus_cd.dropna(subset=["start"])
mus_musculus_cd["end"] = mus_musculus_cd["end"].map(int)
mus_musculus_cd["start"] = mus_musculus_cd["start"].map(int)

In [18]:
mus_musculus_aca["start"] = [int(aca_mm9_2_mm10.ix[i, 1]) if i in aca_mm9_2_mm10.index
                            else np.nan for i in mus_musculus_aca.snor_id]
mus_musculus_aca["end"] = [int(aca_mm9_2_mm10.ix[i, 2]) if i in aca_mm9_2_mm10.index
                            else np.nan for i in mus_musculus_aca.snor_id]
mus_musculus_aca = mus_musculus_aca.dropna(subset=["start"])
mus_musculus_aca["start"] = mus_musculus_aca["start"].map(int)
mus_musculus_aca["end"] = mus_musculus_aca["end"].map(int)

In [19]:
mus_musculus_cd.to_csv("mouse/mmu_cd_box_snoRNAs_mm10.tab",
                       index=False,
                       header=False,
                       sep="\t",
                       na_rep="NaN")
mus_musculus_aca.to_csv("mouse/mmu_haca_box_snoRNAs_mm10.tab",
                       index=False,
                       header=False,
                       sep="\t",
                       na_rep="NaN",
                        encoding='utf-8')

##Comments

###First bugfix

Some of the xml files required adjustment for the sequence and coordinates. The database is not complete. In the end I decided to loosen the requirement for the snoRNAs that snoRNA sequence length has to match snoRNA coordinates length. This solution was faster and as I need the coordinates only for expression estimation is valid. Moreover, some coordinates were provided in hg19 and some with hg18.
 - SNORD11B
 It did not have any coordinates. I took them from hadi and exchanged them.
I exchanged coordinates (adjusted them to the sequence) and added chrom
 - SNORD19B
 Coordinates where wrong (totally different than in UCSC)
 - SNORD105B
 There was no coordinates provided
 - SNORD111B
 Coordinates were wrong
 - SNORD12B
 There was no coordinates
 - SNORD127
 There was no coordinates
 - SNORD123
 There is no such snoRNA in snOPY
 - SNORD1B
 Coordinates were wrong
 - SNORD14E
 no such a snoRNA in snOPY
 - SNORD32B
 wrong coordinates (from hg18)
 - SNORD1C
 wrong coordinates (from hg18)
 - SNORD4B
 wrong coordinates (from hg18)
 - SNORD36B
 wrong coordinates (from hg18)
 - SNORD11
 no such a gene (only snord11b present)
 - SNORD35B
 wrong coordinates (from hg18)
 - SNORD56B
 no such snoRNA
 - SNORD96B
 wrong coordinates (from hg18)
 - SNORD10
 no genomic coordinates provided
 - SNORD42B
 wrong coordinates (from hg18)
 - SNORD109B
 no coordinates provided
 - SNORD62B
 wrong coordinates (from hg18)
 - SNORD54
 no coordinates provided
 - SNORD12C
 wrong coordinates (from hg18)
 - SNORD58B
 wrong coordinates (from hg18)
 - SNORD45B
 wrong coordinates (from hg18)
 - SNORD45C
 wrong coordinates (from hg18)
 - SNORD38B (Homo_sapiens300664.xml)
 there was 3 different genes for this snoRNAs in snOPY. One of them was consistent with Hadi's
 coordinates (however in hg18 so I changed to hg19)
 - SNORD15B
 wrong coordinates (from hg18)
 - SNORD14A (Homo_sapiens300737.xml)
 There were two genes. As in the case of 38B I chosen these one with the consistent chromosome
 to Hadi's result
 - SNORD14C
 no such snoRNA
 - SNORD14D
 no such snoRNA
 - SNORD13
 no such snoRNA
 - snord50B
 wrong coordinates (from hg18)
 - SNORD18B
 wrong coordinates (from hg18)
 - SNORD18C (Homo_sapiens300521.xml)
 Three options were obtained. Chosen as previously.
 - SNORD5
 This one had even chromosome wrongly assigned.
 - SNORD83B
 wrong coordinates (from hg18)
 - SNORD91B
 wrong coordinates (from hg18)
 - SNORD49B
 wrong coordinates (from hg18)
 - SNORD103B
 wrong coordinates (from hg18)
 - SNORD59B
 wrong coordinates (from hg18)
 - SNORD121B
 no such snoRNA
 - SNORD88C
 wrong coordinates (from hg18)
 - SNORD88B
 wrong coordinates (from hg18)
 - SNORD121A
 no such snoRNA
 - SNORD124
 no such snoRNA
 - SNORD125
 no such snoRNA
 - SNORD58C
 wrong coordinates (from hg18)
 - SNORD126
 no such snoRNA
 - SNORD108
 wrong coordinates (from hg18)
 - SNORD97
 no such snoRNA


Those are only snoRNAs that do not have "-" in the name ie. without known subform.

Additionally following snoRNAs did not have coordinates:
 - Homo_sapiens300722    SNORA11B_sapiens300722
 - Homo_sapiens300762     SNORD11_sapiens300762
 - Homo_sapiens300764     SNORA34_sapiens300764
 - Homo_sapiens300765     SNORA78_sapiens300765
 - Homo_sapiens300767    SNORA36B_sapiens300767
 - Homo_sapiens300775    SCARNA14_sapiens300775

and they were filled by hand.

###Second bugfix

There is disagreement between some of the targets found and database:
 - position 1805 (in snOPY) for RNA18S modified by SNORD20:
we find the modification on the position 1806:
RNA18S	1792	1832	1625944_1:SNORD20_sapiens300509:8	34	+	SNORD20_sapiens300509	AGACGGTCGAACTTGACTATCTAGAGGAAGTAAAAGTCGT	-32.0	((((((((((((((((((((.&))))))))))))))))))))	AACTTmGACTATCTAGAGGAAG&TTCCTCTAGATAGTCAAGTT	D	1806	0.0	47454.617287

According to snoOPY this is 1805: GAACTmTGACT but according to snoRNABase and [1] this is as in our prediction.


 - The same situation occurs in the position 1703 (in snOPY) in RNA18S modified by SNORD43. According to our data the modified position
 is 1705:
RNA18S	1682	1722	11405452_1:SNORD43_sapiens300686:N	30	+	SNORD43_sapiens300686	TCCCTGCCCTTTGTACACACCGCCCGTCGCTACTACCGAT	-21.9	.(((((((((.&.))))))))).	ACCGCmCCGTCG&TGACGGGCGGA	D'	1705	0.0	20878.6803211

 What is confirmed in [1] and in snoRNABase.


 - The same for position 1678 modified by SNORD82 in RNA18S in snOPY (according to to snOPY modified nuc is T in sequence but A in snoRNA entry!!!).
 Our data says that it is position 1680 and the nucleotide is A:
RNA18S	1657	1697	2868101_1:SNORD82_sapiens300510:26	52	+	SNORD82_sapiens300510	GGGTCATAAGCTTGCGTTGATTAAGTCCCTGCCCTTTGTA	-16.6	.(((((((((((.&.))))))))))).	GATTAmAGTCCCTG&AAGGGACTTAATA	D'	1680	0.0	72860.9418862

This position (according to the sequence) is confirmed in snoRNABase.

 - CD boxes of SNORD25 are wrongly annotated: they should be 59 and 34 for D and D' box respectively, instead of 60 and 35. This mistakes causes that predicted 
 nucleotide modification is 1489 and the true modification site is located 1490 (of course the modified nucleotide is also wrong).


Conclusions:
 - There is a lot of mistakes in the snOPY database. There is bad modification position annotation and mixed genomes used.

##Bibliography

1. Kiss-László, Z., Henry, Y., Bachellerie, J. P., Caizergues-Ferrer, M. & Kiss, T. Site-specific ribose methylation of preribosomal RNA: a novel function for small nucleolar RNAs. Cell 85, 1077–1088 (1996).

##Utils

In [2]:
%matplotlib inline
import re
import glob
import swalign
import pandas as pd
import numpy as np
import pylab as pl
import seaborn as sns
from random import choice
import xml.etree.ElementTree as ET
from collections import OrderedDict
from Bio import SeqIO
from IPython.display import Image
pl.rcParams['figure.figsize'] = (14, 10)
pl.rcParams['ytick.labelsize'] = 20
pl.rcParams['xtick.labelsize'] = 20
pl.rcParams['axes.labelsize'] = 23
pl.rcParams['legend.fontsize'] = 20
sns.set_style('ticks')
c1, c2, c3 = sns.color_palette("Set1", 3)
from IPython.core.display import HTML, Image
css_file = '/import/bc2/home/zavolan/gumiennr/.ipython/profile_default/static/custom/custom.css'
HTML(open(css_file, "r").read())

In [3]:
def apped_assembly(df):
    #
    # prepare swalign parameters
    #
    scoring = swalign.NucleotideScoringMatrix(match=2,
                                              mismatch=-5)
    sw = swalign.LocalAlignment(scoring,
                                gap_penalty=-6,
                                gap_extension_penalty=-4)
    hg19_score = []
    hg18_score = []
    assembly = []
    for idx, row in df.iterrows():
        try:
            h19 = sw.align(row[6], row[7]).score
        except AttributeError:
            h19 = 0.0
        try:
            h18 = sw.align(row[6], row[8]).score
        except AttributeError:
            h18 = 0.0
        hg19_score.append(h19)
        hg18_score.append(h18)

        if h19 > h18:
            assembly.append("hg19")
        elif h19 < h18:
            assembly.append("hg18")
        else:
            assembly.append("ambiguous")
    df["hg19score"] = hg19_score
    df["hg18score"] = hg18_score
    df["assembly"] = assembly
    return df

In [4]:
def apped_mouse_assembly(df):
    #
    # prepare swalign parameters
    #
    scoring = swalign.NucleotideScoringMatrix(match=2,
                                              mismatch=-5)
    sw = swalign.LocalAlignment(scoring,
                                gap_penalty=-6,
                                gap_extension_penalty=-4)
    mm9_score = []
    mm10_score = []
    assembly = []
    for idx, row in df.iterrows():
        try:
            mm10 = sw.align(row[6], row[7]).score
        except AttributeError:
            mm10 = 0.0
        try:
            mm9 = sw.align(row[6], row[8]).score
        except AttributeError:
            mm9 = 0.0
        mm10_score.append(mm10)
        mm9_score.append(mm9)

        if mm10 > mm9:
            assembly.append("mm10")
        elif mm10 < mm9:
            assembly.append("mm9")
        else:
            assembly.append("ambiguous")
    df["mm10score"] = mm10_score
    df["mm9score"] = mm9_score
    df["assembly"] = assembly
    return df